# 05_ex

1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

--2022-11-30 15:37:32--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/xvjzaxzz3ysphme/data_000637.txt [following]
--2022-11-30 15:37:33--  https://www.dropbox.com/s/raw/xvjzaxzz3ysphme/data_000637.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc060c91d4cef9788d8543f76b55.dl.dropboxusercontent.com/cd/0/inline/BxtysWzZcUygATzgxJUO9Yly9x_h2agtqr5j-HD-DHAPFjoSYIfEbzkF7R8i15oPYZTjLoFTz_lzaZoNpV8SINW3ZpVxSa11wuM27OxsK2aQJry7HI_YjEaAo8rUUYvx_EfNYqo1ZLMpMrpllQR3IgENLnbNrPgVJeT2eMcgxRxZqw/file# [following]
--2022-11-30 15:37:34--  https://uc060c91d4cef9788d8543f76b55.dl.dropboxusercontent.com/cd/0/inline/BxtysWzZcUygATzgxJUO9Yly9x_h2agtqr5j-HD-DHAPFjoSYIfEbzkF7R8i15oPYZTj

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('data/data_000637.txt',nrows=15000)
data_full = pd.read_csv('data/data_000637.txt')
data.head()
df = pd.DataFrame(data)
df_full = pd.DataFrame(data_full)

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [4]:
BX_max_value = df_full.BX_COUNTER.max()
print("Max BX number in a orbit: "+str(BX_max_value))

Max BX number in a orbit: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [5]:
import datetime as dt

df['TIMENS'] = data['TDC_MEAS'] * 25 / 30 + data['BX_COUNTER'] * 25

data

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS
0,1,0,123,3869200167,2374,26,59371.666667
1,1,0,124,3869200167,2374,27,59372.500000
2,1,0,63,3869200167,2553,28,63848.333333
3,1,0,64,3869200167,2558,19,63965.833333
4,1,0,64,3869200167,2760,25,69020.833333
...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,84982.500000
14996,1,1,17,3869200316,3400,15,85012.500000
14997,1,1,10,3869200316,3530,16,88263.333333
14998,1,1,8,3869200316,3533,18,88340.000000


In [6]:
s = pd.Series(pd.to_timedelta(df['TIMENS'], unit='ns')) 
df['TOTAL_TIME(sec)'] = s.dt.total_seconds()

df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
0,1,0,123,3869200167,2374,26,59371.666667,0.000059
1,1,0,124,3869200167,2374,27,59372.500000,0.000059
2,1,0,63,3869200167,2553,28,63848.333333,0.000064
3,1,0,64,3869200167,2558,19,63965.833333,0.000064
4,1,0,64,3869200167,2760,25,69020.833333,0.000069
...,...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,84982.500000,0.000085
14996,1,1,17,3869200316,3400,15,85012.500000,0.000085
14997,1,1,10,3869200316,3530,16,88263.333333,0.000088
14998,1,1,8,3869200316,3533,18,88340.000000,0.000088


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [15]:
for i in range(1,len(df['TIMENS'])):
    df['TIMENS'][i]+=df['TIMENS'][i-1]

/tmp/ipykernel_6318/1225524176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TIMENS'][i]+=df['TIMENS'][i-1]


In [14]:
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
0,1,0,123,3869200167,2374,26,5.937167e+04,0.000059
1,1,0,124,3869200167,2374,27,2.968592e+05,0.000059
2,1,0,63,3869200167,2553,28,8.950550e+05,0.000064
3,1,0,64,3869200167,2558,19,2.100518e+06,0.000064
4,1,0,64,3869200167,2760,25,4.228826e+06,0.000069
...,...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,9.473098e+19,0.000085
14996,1,1,17,3869200316,3400,15,9.475631e+19,0.000085
14997,1,1,10,3869200316,3530,16,9.478165e+19,0.000088
14998,1,1,8,3869200316,3533,18,9.480699e+19,0.000088


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [4]:
import numpy as np

df.groupby('TDC_CHANNEL').max()[-3:]

,HEAD,FPGA,ORBIT_CNT,BX_COUNTER,TDC_MEAS
TDC_CHANNEL,,,,,
127,1,1,3869200301,3337,29
128,1,1,3869200306,3337,23
139,1,1,3869200316,3560,0


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [5]:
df_orbit = df.groupby('ORBIT_CNT').count()
print(df_orbit)

            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS
ORBIT_CNT                                                
3869200167    43    43           43          43        43
3869200168    85    85           85          85        85
3869200169   127   127          127         127       127
3869200170    98    98           98          98        98
3869200171   109   109          109         109       109
...          ...   ...          ...         ...       ...
3869200312    90    90           90          90        90
3869200313    75    75           75          75        75
3869200314    93    93           93          93        93
3869200315   136   136          136         136       136
3869200316    82    82           82          82        82

[150 rows x 5 columns]


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [7]:
tdc_139 = df[(df['TDC_CHANNEL'])==139].groupby('ORBIT_CNT').count()
print(1, len(df))
print(11, len(df[(df['TDC_CHANNEL'])==139]))
print(tdc_139)

1 15000
11 1268
            HEAD  FPGA  TDC_CHANNEL  BX_COUNTER  TDC_MEAS
ORBIT_CNT                                                
3869200167     4     4            4           4         4
3869200168     8     8            8           8         8
3869200169    11    11           11          11        11
3869200170     9     9            9           9         9
3869200171     9     9            9           9         9
...          ...   ...          ...         ...       ...
3869200312     8     8            8           8         8
3869200313     7     7            7           7         7
3869200314    10    10           10          10        10
3869200315     7     7            7           7         7
3869200316     8     8            8           8         8

[150 rows x 5 columns]


In [8]:
tdc_139 = df[(df['TDC_CHANNEL'])==139]
count_unique_139 = len(set(tdc_139["ORBIT_CNT"]))
print("Number of unique orbits with at least one measurement from TDC_CHANNEL=139: " + str(count_unique_139))
tdc_139

Number of unique orbits with at least one measurement from TDC_CHANNEL=139: 150


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
7,1,0,139,3869200167,2776,0
15,1,1,139,3869200167,2797,0
30,1,0,139,3869200167,3085,0
39,1,0,139,3869200167,3186,0
48,1,0,139,3869200168,23,0
...,...,...,...,...,...,...
14956,1,0,139,3869200316,285,0
14970,1,0,139,3869200316,1001,0
14985,1,0,139,3869200316,3389,0
14989,1,1,139,3869200316,3398,0


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [9]:
fpga_0 = df[(df['FPGA'])==0]

fpga_1 = df[(df['FPGA'])==1]

print(fpga_0.groupby('TDC_CHANNEL').count())
print(fpga_1.groupby('TDC_CHANNEL').count())

             HEAD  FPGA  ORBIT_CNT  BX_COUNTER  TDC_MEAS
TDC_CHANNEL                                             
1               8     8          8           8         8
2              16    16         16          16        16
3              16    16         16          16        16
4              19    19         19          19        19
5              19    19         19          19        19
...           ...   ...        ...         ...       ...
121            63    63         63          63        63
122            71    71         71          71        71
123           202   202        202         202       202
124           193   193        193         193       193
139           879   879        879         879       879

[117 rows x 5 columns]
             HEAD  FPGA  ORBIT_CNT  BX_COUNTER  TDC_MEAS
TDC_CHANNEL                                             
1             338   338        338         338       338
2             363   363        363         363       363
3      

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.